# A MILP és az LP solverek összehasonlítása

## Importok és globális változók definiálása

In [ ]:
import pandas as pd
import numpy as np
import vppopt as vo
import sys
import compare as cp
from matplotlib import pyplot as plt

np.set_printoptions(suppress=True, linewidth=100000, threshold=sys.maxsize)
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',504)
pd.set_option('display.width',1000)
# Napelem zajosításához
seed = 1000

# Kapcsoló az ábrák megjelenítésére 
cp.globaldisplay=True
# kapcsoló az összegzések elvégzéséhez
cp.globalsummerize=False


T = 24
l1 = vo.gen_random_demand(T, 0, 1000, 12200)
l2 = vo.gen_random_demand(T, 0, 2000, 4562)
l3 = vo.gen_random_demand(T, 0, 3000, 7531)
l4 = vo.gen_random_demand(T, 0, 10000, 891234)
l5 = vo.gen_random_demand(T, 2000, 4000, 97345)
l6 = vo.gen_random_demand(T, 5000, 6000, 895)
l7 = vo.gen_random_demand(T, 500, 1000, 12)
l8 = vo.gen_random_demand(T, 0, 20000, 7533)
l9 = vo.gen_random_demand(T, 10000, 20000, 9453)
l10 = np.array([4625, 4420, 4250, 4190, 4269, 4480, 4960, 4931, 4887, 4930, 5063, 5239, 5394, 5551, 5174, 5659, 5137, 5630, 5055, 5444, 4962, 5177, 4932, 4935])
d = np.zeros(T)

### Fogyasztási görbék megjelenítése

In [ ]:
plt.plot(l1)
# plt.plot(l2)
# plt.plot(l3)
# plt.plot(l4)
# plt.plot(l5)
# plt.plot(l6)
plt.plot(l7)
# plt.plot(l8)
# plt.plot(l9)
# plt.plot(l10)
plt.show()
pd.DataFrame([l1, l2, l3, l4, l5, l6, l7, l8, l9, l10], columns=range(1,25))

# Tesztek a solverek érzékenységére

## LP solver

### Fel- és lefutás érzékenysége (Gázmotoroknál)

#### Csak gázmotorok

**1. Teszt:**<br>
2-2 gázmotorunk van, amelyek közül a második pár 2. tagjánál módosítjuk a ramp up/down rate-et.
Csak az *l1, l2, l3, l7* fogyaztói görbéket használjuk, mivel a többi nem teljesíthető. Ezen teszt esetén a ramp up és down értékeket csak csökkentjük, növelés nem történik.

**Eredmény:**
A vizsgált fogyasztási görbékre nézve nem tapasztalható a költségben bármiféle változás, viszont a termelés görbékben annál inkább. Ezen esetekben a tesztek többsége során jelentős elmozdulás az első 5%-ban, illetve 30-35%-os csökkentés esetén volt tapasztalható.
Ezen felül a 2. és 3. vizsgálat esetén 70 és 35%-nál nagyobb csökkentésre már nem optimalizálható a modell, így az ottani paramétereket módosítva a következő tesztben nézzük meg.

In [ ]:
# ====== Ne módosítsd =======
cp.globalcurrcat = 0
# ===========================

def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)


_ = cp.sensitivity([g1_1,g1_2], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Az 1. teszt módosított verziója, hogy a 3. vizsgálat is teljes egészében optimalizálható legyen.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)


_ = cp.sensitivity([g1_1,g1_2], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 625),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2 Teszt:**<br>
Még egy kisebb módosítás, amely a költségeket érinti. Jelen esetben meg lett 10-szerezve az első gázmotor költsége és az olcsóbb gázmotor ramp up/down értékeit csökkentjük.<br>
**Eredmény:** Ezzel a módosítással már elég érdekes eremények születtek. Ahogy látható, akár az első tesz esetén is már a költségfüggvény változásnak indult. Ez a változás ránézésre a konstans 0-ból egy exponenciális függvény.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)

_ = cp.sensitivity([g1_1,g1_2], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 625),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.3. Teszt:**<br>
Annyi módosítással az 1.2-höz képest, hogy 1500 helyett minden gázmotornak 750 az alap ramp up/down értéke.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 100), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 100)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 10)

_ = cp.sensitivity([g1_1,g1_2], f, [(750, 750)], [(750, 593),(750, 750)], T, [l1, l7], [np.zeros(T)])

**1.3.1 Teszt:**<br>
Az elöző teszteset részletesebb vizsgálata. (1. eset csökkentés)

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 750, 750, 100), vo.VPPGasEngine(1500, val, val, 10)]
cp.sensitivity_by_participants(f, 190, 1100, 5, l1, d, T, view=(30,250))

**2. Teszt**<br>
Növeljük meg a maximális termelésüket a motoroknak és ezzel az alap ramp up/down rátát is.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10), vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10)

_ = cp.sensitivity([g1_1,g1_2], f, [(2000, 4000)], [(2000, 2000),(2000, 2000),(2000, 1677),(2000, 2000),(2000, 2000),(2000, 2000),(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.1 Teszt:**<br>
Legyen drágább a nagyobb max. termeléssel bíró gázmotor.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10), vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 100)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 100)

_ = cp.sensitivity([g1_1,g1_2], f, [(2000, 4000)], [(2000, 2000),(2000, 2000),(2000, 1677),(2000, 2000),(2000, 2000),(2000, 2000),(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.2 Teszt:**<br>
Most a másik motor legyen drágább.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 100), vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 100)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10)

_ = cp.sensitivity([g1_1,g1_2], f, [(2000, 4000)], [(2000, 2000),(2000, 2000),(2000, 1677),(2000, 2000),(2000, 2000),(2000, 2000),(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**Összefogalaló:**<br>
Az előbbi tesztek esetén amennyiben azonos költségekkel futottak a motorok azon esetekben a költség nem változott, viszont amint különbség volt közöttük, egyből exponeneciális változás volt tapasztalható.
Termelés tekintetében is a költséghez van köze a függvény alakjának, ha a költségek megegyeznek, akkor egy kesze-kusza függvényt kapunk, viszont amint különböznek, ez is átvált exponenciálisba.

#### Gázmotorok + energia tároló

**1. Teszt:**<br>
Úgy, hogy bejött a tároló, az eddig nem teljesíthető optimalizáciokat is el lehetett végezni, viszont ennek következtében az ottani költségek hatalmasat ugrottak.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10), vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1,g1_2, t], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Visszatérve a tároló nélkül is teljesíthető paraméterekre. Ezen eredmények igazolják az előbbi tesztnél leírtakat.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10), vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)


_ = cp.sensitivity([g1_1,g1_2, t], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 625),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2 Teszt:**<br>
Növeljük meg az egyik gázmotor költségét.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10),vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)

_ = cp.sensitivity([g1_1,g1_2,t], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 625),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2.1 Teszt:**<br>
Most helyezzük vissza az összes paramétert úgy, hogy azok 1500-nak százalékos értékével csökkentsék az alap értéket.<br>
**Eredmény:** Nem tapasztalható releváns eltérés a grafikonok között. Ránézésre olyan, mintha nagyobb intervallumon vennénk a függvényeket.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10),vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)

_ = cp.sensitivity([g1_1,g1_2,t], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.3. Teszt:**<br>
Csökentjük az alap ramp up/down értékeket. (1500 $\rightarrow$ 750)

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 100), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10),vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 100)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 10)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)

_ = cp.sensitivity([g1_1,g1_2,t], f, [(750, 750)], [(750, 750)], T, [l1, l2, l7], [np.zeros(T)])

**2. Teszt**<br>
Növeljük meg a maximális termelésüket a motoroknak és ezzel az alap ramp up/down rátát is.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10), vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10),vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)

_ = cp.sensitivity([g1_1,g1_2,t], f, [(2000, 4000)], [(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.1 Teszt:**<br>
Növeljük meg a második gázmotor költségét.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10), vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 100),vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 100)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)

_ = cp.sensitivity([g1_1,g1_2,t], f, [(2000, 4000)], [(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.2 Teszt:**<br>
Most cseréljük fel a költségeket.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 100), vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10),vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 100)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)

_ = cp.sensitivity([g1_1,g1_2,t], f, [(2000, 4000)], [(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**Összefoglalás:**<br>
Összességében hasonló görbéket kapunk, mintha csak gázmotorok szerepelnének, csak kevésbé szögleteseket, a tárolónak köszönhetően sokkal "görbébb" függvényeket kapunk. Ezek mellett sokszor kisebbek a hibák, mintha csak gázmotorok lennének, illetve sok esetben az eddig nem teljesíthető teszteket is sikerül elvégeznie.

#### Gázmtorok + napelem

**1. Teszt:**<br>
Adjunk egy napelemet a 2 gázmotorhoz úgy, hogy annak termelését arányosra állítjuk a gázmotorokéval. 

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10),vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Módosítjuk a paramétereket, hogy minden esetben elvégezhető legyen az optimalizálás.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10),vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 945),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2 Teszt:**<br>
Kis költség változtatás.<br>
**Eredmény:** Hasonló az eredmény, mint az eddigieknél. (Exponenciális költség növekedés.)

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10),vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 945),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2.1 Teszt:**<br>
Paraméterek visszaállítása az 1. tesztbeli értékekre.
Itt most nincs különbség a kettő között, nem mint a tárolók esetén...

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10),vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2.2 Teszt:**<br>
Játszunk a költséggel. Lehet, hogy azért exponeneciális a költség különbség függvénye, mert a költségek is így viszonyulnak egymáshoz?<br>
**Eredmény:** Az alábbi grafikonok alapján téves a feltevés, mivel így is exponeneciálisak lesznek.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 20), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10),vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 20)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 945),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.3 Teszt:**<br>
Módosítsuk az alap ramp up/down értékeket az eddigi felére. (1500 $\rightarrow$ 750)

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 10), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10),vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 10)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 10)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(750, 750)], [(750, 750)], T, [l1, l7], [np.zeros(T)])

**2. Teszt:**<br>
Növeljük meg a gázmotorok maximális termelését.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10), vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10),vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10)
s = vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(2000, 4000)], [(2000, 2000),(2000, 2000),(2000, 1997),(2000, 2000),(2000, 2000),(2000, 2000),(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.1 Teszt:**<br>
Legyen drágább a nagyobb max. termeléssel bíró gázmotor.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10), vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 100),vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 100)
s = vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(2000, 4000)], [(2000, 2000),(2000, 2000),(2000, 1997),(2000, 2000),(2000, 2000),(2000, 2000),(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.2 Teszt:**<br>
Most a másik motor legyen drágább.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 100), vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10),vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 100)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10)
s = vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(2000, 4000)], [(2000, 2000),(2000, 2000),(2000, 1997),(2000, 2000),(2000, 2000),(2000, 2000),(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**Összefogalalás:**
Többnyire hasonló görbéket kapunk, mint a sima gázmotoroknál, csak a legtöbb esetben sokkal kacifántosabbak. Ezen felül előfordul olyan eset, melyet ez a modell teljesíteni tud, míg a sima gázmotoros nem, mely várható volt a termelt energia mennyiségének növekedése végett.

#### Gázmotork + napelem + enegia tároló

**1. Teszt:**<br>
Az eddigi résztvevőket kombinálva most 2 gázmotorral, 1 tárolóval és 1 napelemmel fogjuk az optimalizációt végezni.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10), vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0), vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)


_ = cp.sensitivity([g1_1,g1_2, t, s], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Megváltoztatjuk a költségeket a gázmotorok esetén.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10), vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0), vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)


_ = cp.sensitivity([g1_1,g1_2, t, s], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2 Teszt:**<br>
Módosítsuk az alap ramp up/down értékeket az eddigi felére. (1500 $\rightarrow$ 750)

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 100), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10), vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0), vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 100)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 10)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)


_ = cp.sensitivity([g1_1,g1_2, t, s], f, [(750, 750)], [(750, 750)], T, [l1, l2, l3, l7], [np.zeros(T)])

**2. Teszt:**<br>
Növeljük meg a gázmotorok maximális termelését.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10), vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10), vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0), vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
s = vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)


_ = cp.sensitivity([g1_1,g1_2, t, s], f, [(2000, 4000)], [(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.1 Teszt:**<br>
Legyen drágább a nagyobb max. termeléssel bíró gázmotor.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10), vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 100), vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0), vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 100)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
s = vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)


_ = cp.sensitivity([g1_1,g1_2, t, s], f, [(2000, 4000)], [(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.2 Teszt:**<br>
Most a másik motor legyen drágább.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 100), vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10), vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0), vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 100)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
s = vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)


_ = cp.sensitivity([g1_1,g1_2, t, s], f, [(2000, 4000)], [(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**Összefogalalás:**
Itt is többnyire hasonló görbéket kapunk, mint a tárolók esetén, viszont még ívesebbeket, kevesebb "hegye" van a görbéknek. Ezen felül ebben az esetben a hibák hamarabb kisebb változtatások esetén is már kezdenek előjönni, viszont kisebbek, mint a többi esetén. Hasonló mondható el a költségekről is, hamarabb nőnek, viszont sokkal kevesebbet.

### Költség érzékenysége (Gázmotoroknál)

#### Csak gázmotorok

**1. Teszt:** <br>
2-2 gázmotorral végezzük ezt a tesztet, és azok közül az egyik költségét változtatjuk.<br>
**Eredmény:** Ahogy azt a grafikonokon is láthatjuk, a paraméter változtatására vagy szakaszonként, vagy teljesen lineáris változást tapasztalhatunk a költségfüggvényben.

In [ ]:
# ========= Ne módosítsd! ==========
cp.globalcurrcat = 1
# ============================
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 1500, 1500, val[0]+i*val[1])]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 50)


_ = cp.sensitivity([g1_1,g1_2], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Vizsgáljuk meg azt az esetet, amikor a változtatott költségű gázmotor fel- és lefutási értékeit megfelezzük.<br>
**Eredmény:** Sok különbség az elöző esethez képest nincsen a költségeket tekintve, viszont itt már a termelésben észlelhetünk különbségeket, de többnyire konstans hibákkal. Ezen felül a 3. teszteset itt már nem teljesíthető.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 1500, 1500, val[0]+i*val[1])]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 50)


_ = cp.sensitivity([g1_1,g1_2], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**2. Teszt:**<br>
Most növeljük meg a gázmotorok maximális termelését és vele együtt a fel- és lefutási ráták értékeit.<br>
**Eredmény:** sok különbség ez esetben sincs az eddigi ereményekhez képest. Egyedül az hozható fel, hogy ez több esetet képes teljesíteni.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(6000, 3000, 3000, 500), vo.VPPGasEngine(4000, 2000, 2000, val[0]+i*val[1])]
g1_1 = vo.VPPGasEngine(6000, 3000, 3000, 500)
g1_2 = vo.VPPGasEngine(4000, 2000, 2000, 500)


_ = cp.sensitivity([g1_1,g1_2], f, [(500, 500)], [(500, 500)], T, [l1, l2, l3, l5, l6, l7, l10], [np.zeros(T)])

#### Gázmotorok + tároló 

**1. Teszt:** <br>
A teszt során 2 gázmotor és egy energia tároló segítségével oldjuk meg a feladatokat. <br>
**Eredmények:** Az elöző szekció 1 tesztjéhez hasonlítva itt jelentős eltérések vehetők észre. Ilyen például a 2. teszt esetén 50-55% közötti érdekes ugrás a termelések hibájában, melyet a következő teszt során mélyebben megvizsgálok. Összességében viszont az mondható el, hogy olcsóbbban termeli meg a modell a szükséges áramot. 

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 1500, 1500, val[0]+i*val[1]), vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 50)
s = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1, g1_2, s], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Ezen teszt során egy megadott intervallumon változtatva a költség paraméterét részletesen, részvevőkre lebontva elemezem a termelést.<br>
**Eredmények:** Megvizsgálva a termelési görbéket szépen látszik, ahogy a költséget változtatjuk azok miként alakulnak. Ezen az intervallumon 3 ponton történik változás, melyek rendre 17 és 18 között, 20 és 21 között, valamint 22 és 23 között. Ha összevetjük az elöző teszt 2. esetével jól látszik, ahogyan a költség függvény változik ott mi történik a termelésben


In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 1500, 1500, val), vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50-50*0.7, 24, 10, l2, d, T)

**1.2 Teszt:**<br>
Másik intervallumra is megnézve.<br>
**Eredmények:** Itt csak 1 töréspont vehető észre.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 1500, 1500, val), vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50+50*0.6,50+50*0.7, 10, l2, d, T)

**1.3 Teszt:**<br>
Nézzük meg másik fogyasztási görbére is.<br>
**Eredmények:** Itt is csak 1 ilyen pont található.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 1500, 1500, val), vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50+50*0.65,50+50*0.75, 10, l3, d, T, (30,150))

**2. Teszt:**<br>
Vizsgáljuk meg azt az esetet, amikor a változtatott költségű gázmotor fel- és lefutási értékeit megfelezzük<br>
**Eredmények:** A sima gázmotoros esethez képest sokkal érdekesebb eredményeket kaptunk. Sok esetben nőttek azon pontok száma, ami után a modell termelési stratégiát vált. Következő tesztek ezek részletesebb vizsgálatáról fognak szólni.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 750, 750, val[0]+i*val[1]), vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 50)
s = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1, g1_2, s], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**2.1 Teszt:**<br>
Elöző teszt 1. vizsgálatának részletesebb megtekintése.<br>
**Eredmények:** Szépen látszik a grafikonon, hogy egy idő után már érdemes bekapcsolni az 1-es gázmotort, mert úgy olcsóbb a termelés.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 750, 750, val), vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50-50*0.45,50-50*0.5, 10, l1, d, T, (30,40), addcontour=False)

**2.2 Teszt:**<br>
Ezöző teszt 2. vizsgálatának részletesebb megtekintése.<br>
**Eredmények:** It is szépen látszik a grafikonokon, hogy hol változnak az értékek itt is a változások száma megegyezik a grafikonon láthatóéval.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 750, 750, val), vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50-50*0.55,50-50*0.25, 10, l2, d, T, (30,40), addcontour=False)

**3. Teszt:**<br>
Növeljük meg a termelési képességüket a gázmotoroknak.<br>
**Eredmények:** Az eredményekben nincs semmi érdekes. A hibák konstansok, a költség lineárisan csökken/nő.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(6000, 3000, 3000, 500), vo.VPPGasEngine(4000, 2000, 2000, val[0]+i*val[1]),vo.VPPEnergyStorage(0, 10000, 2000, 2000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(6000, 3000, 3000, 500)
g1_2 = vo.VPPGasEngine(4000, 2000, 2000, 500)
s = vo.VPPEnergyStorage(0, 10000, 2000, 2000, 0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1,g1_2,s], f, [(500, 500)], [(500, 500)], T, [l1, l2, l3, l5, l6, l7, l10], [np.zeros(T)])

#### Gázmotorok + napelem

**1. Teszt:**<br>
Ezen tesztek során 2 gázmotor és egy napelem lesz a részvevőnk.<br>
**Eredmények:** A grafikonokon 1 érdekes jelenség van csak. A csökkentés esetén az utolcsó -100%-ra a hiba megugrik.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 1500, 1500, val[0]+val[1]*i),vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 50)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Most az elöző teszt esetén említett felugrást vizsgálom.<br>
**Eredmények:** Mivel addigra a gázmotor költsége 0 lesz, így inkább kitermeli ingyen az energiát, minthogy a napelemét használja 1 költségért.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 1500, 1500, val), vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
cp.sensitivity_by_participants(f, 0,50-50*0.95, 2, l1, d, T)

**2. Teszt:**<br>
Módosítsuk most a változtatott gázmotor fel- és lefutási értékeit a felére.<br>
**Eredmények:** Hasonló eseteket tapasztalhatunk mintha csak gázmotorok lennének a teszben. Viszont a termelési hibák amikor változnak akkor azt kisebb mértékben teszik.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 750, 750, val[0]+val[1]*i),vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 50)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**3. Teszt:**<br>
Növeljük meg a termelési képességüket a gázmotoroknak.<br>
**Eredmények:** Nincs semmi érdekes sajnos. Ugyan azt kaptuk, mint az eredeti gázmotorosnál, csak itt vannak hibák is és olcsóbb a termelés a napelem miatt.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(6000, 3000, 3000, 50), vo.VPPGasEngine(4000, 2000, 2000, val[0]+val[1]*i),vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(6000, 3000, 3000, 50)
g1_2 = vo.VPPGasEngine(4000, 2000, 2000, 50)
s = vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

#### Gázmotorok + napelem + enegia tároló

**1. Teszt:**<br>
A tesztek során 2 gázmotort, 1 tárolót és 1 napelemet alkalmazok.<br>
**Eredmények:** Az első tesztnél már látható, hogy a termelésnél a hiba eléggé hullámos, amit a következő tesztekben jobban meg is vizsgálok.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 1500, 1500, val[0]+val[1]*i),vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed),vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 50)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, so, st], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Az elöző teszteset első vizsgálatának részletesebb tárgyalása.<br>

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 1500, 1500, val), vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed),vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50-50*0.8,50-50*0.4, 4, l1, d, T)

**2. Teszt:**<br>
Módosítsuk a változtatott gázmotor fel- és lefutási rátáját a felére.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 750, 750, val[0]+val[1]*i),vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed),vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 50)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, so, st], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**2.1 Teszt:**<br>
Vizsgáljuk meg a 3. esetet az elöző tesztből.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50), vo.VPPGasEngine(1500, 750, 750, val), vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed),vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50-50*0.55,50-50*0.25, 4, l3, d, T)

**3. Teszt:**<br>
Növeljük meg a gázmotorok termelését.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(6000, 3000, 3000, 50), vo.VPPGasEngine(4000, 2000, 2000, val[0]+val[1]*i),vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed),vo.VPPEnergyStorage(0, 10000, 3000, 3000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(6000, 3000, 3000, 50)
g1_2 = vo.VPPGasEngine(4000, 2000, 2000, 50)
st = vo.VPPEnergyStorage(0, 10000, 3000, 3000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, so, st], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l4, l5, l6, l7, l10], [np.zeros(T)])

**3.1 Teszt:**<br>
Vizsgáljuk meg a 2. esetet az elöző tesztből.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(6000, 3000, 3000, 50), vo.VPPGasEngine(4000, 2000, 2000, val), vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed),vo.VPPEnergyStorage(0, 10000, 3000, 3000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50,50+50*0.05, 3, l2, d, T)

### Max termlés érzékenysége (Gázmotoroknál)

#### Csak gázmotorok

**1. Teszt:**

In [ ]:
cp.globalcurrcat = 2
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(val[0]+val[1]*i, 1500, 1500, 10)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)


_ = cp.sensitivity([g1_1,g1_2], f, [(1500, 1500)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Elöző teszt 2 eset pozitív irányának vizsgálata.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(val, 1500, 1500, 10)]
cp.sensitivity_by_participants(f, 1500,1500+1500*0.05, 2, l2, d, T, addcontour=False)

**2. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100), vo.VPPGasEngine(val[0]+val[1]*i, 3000, 3000, 10)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100)
g1_2 = vo.VPPGasEngine(1500, 3000, 3000, 10)


_ = cp.sensitivity([g1_1,g1_2], f, [(1500, 7000)], [(1500, 1500)], T, [l1, l2, l3,l7], [np.zeros(T)])

**2.1 Teszt:**<br>
Elöző teszt 1 esete csökkentésnél.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100), vo.VPPGasEngine(val, 3000, 3000, 10)]
cp.sensitivity_by_participants(f, 0, 1500-1500*0.35, 10, l1, d, T)

**2.2 Teszt:**<br>
Elöző teszt 2 esete növelésnél.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100), vo.VPPGasEngine(val, 3000, 3000, 10)]
cp.sensitivity_by_participants(f, 1500, 1500+7000*0.1, 10, l2, d, T)

**2.3 Teszt:**<br>
Elöző teszt 3 esete növelésnél.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100), vo.VPPGasEngine(val, 3000, 3000, 10)]
cp.sensitivity_by_participants(f, 1500, 1500+7000*0.2, 10, l3, d, T)

#### Gázmotorok + tároló

**1. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(val[0]+val[1]*i, 1500, 1500, 10), vo.VPPEnergyStorage(0,10000, 1000, 1000 ,0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
s = vo.VPPEnergyStorage(0,10000, 1000, 1000 ,0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1,g1_2,s], f, [(1500, 1500)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Elöző teszt 1 eset negatív irány.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(val, 1500, 1500, 10), vo.VPPEnergyStorage(0,10000, 1000, 1000 ,0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 1500-1500*0.7, 1500-1500*0.45, 10, l1, d, T)

**1.2 Teszt:**<br>
Elöző teszt 2 eset negatív irány.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(val, 1500, 1500, 10), vo.VPPEnergyStorage(0,10000, 1000, 1000 ,0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 1500-1500*0.65, 1500-1500*0.45, 10, l2, d, T)

**1.2.1 Teszt:**<br>
Elöző teszt 2 eset negatív irány másik intervallumon.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(val, 1500, 1500, 10), vo.VPPEnergyStorage(0,10000, 1000, 1000 ,0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 1500-1500*0.85, 1500-1500*0.65, 10, l2, d, T)

**2. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100), vo.VPPGasEngine(val[0]+val[1]*i, 3000, 3000, 10), vo.VPPEnergyStorage(0,10000, 1000, 1000 ,0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100)
g1_2 = vo.VPPGasEngine(1500, 3000, 3000, 10)
s = vo.VPPEnergyStorage(0,10000, 3000, 3000 ,0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1,g1_2,s], f, [(1500, 7000)], [(1500, 1500)], T, [l1, l2, l3,l7], [np.zeros(T)])

#### Gázmotorok + napelem

**1. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(val[0]+val[1]*i, 1500, 1500, 10), vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2,so], f, [(1500, 1500)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1. Teszt:**<br>
Elöző teszt 1 estének negatív oldala.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(val, 1500, 1500, 10), vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
cp.sensitivity_by_participants(f, 1500-1500*0.4, 1500-1500*0.3, 3, l1, d, T)

**2. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100), vo.VPPGasEngine(val[0]+val[1]*i, 3000, 3000, 10), vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100)
g1_2 = vo.VPPGasEngine(1500, 3000, 3000, 10)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2,s], f, [(1500, 7000)], [(1500, 1500)], T, [l1, l2, l3,l7], [np.zeros(T)])

#### Gázmotorok + tároló + napelem

**1. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPGasEngine(val[0]+val[1]*i, 1500, 1500, 10), vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0),vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2,st, so], f, [(1500, 1500)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**2. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100), vo.VPPGasEngine(val[0]+val[1]*i, 3000, 3000, 10),vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0),vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100)
g1_2 = vo.VPPGasEngine(1500, 3000, 3000, 10)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2,st, so], f, [(1500, 7000)], [(1500, 1500)], T, [l1, l2, l3,l7], [np.zeros(T)])

### Veszteségi faktor érzékenysége (Tárolók)

**1. Teszt:**

In [ ]:
cp.globalcurrcat = 3
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 10), vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPEnergyStorage(0, 10000, 1000, 1000, val[0] + val[1]*i, val[0] + val[1]*i, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 10)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1,g1_2,st], f, [(.1, .899)], [(.1, .099)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Elöző teszt 3. esetének részletesebb vizsgálata.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 750, 750, 10), vo.VPPGasEngine(1500, 1500, 1500, 10), vo.VPPEnergyStorage(0, 10000, 1000, 1000, val, val, 10, 10, 0)]
cp.sensitivity_by_participants(f, 0.01, 0.5+0.499*1, 3, l3, d, T)

**2. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 1000), vo.VPPGasEngine(1500, 1500, 1500, 1000), vo.VPPEnergyStorage(0, 10000, 1000, 1000, val[0] + val[1]*i, val[0] + val[1]*i, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 1000)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 1000)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1,g1_2,st], f, [(.1, .899)], [(.1, .099)], T, [l1, l2, l3, l7], [np.zeros(T)])

**3. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 1000), vo.VPPGasEngine(1500, 1500, 1500, 1000), vo.VPPEnergyStorage(0, 10000, 1000, 1000, val[0] + val[1]*i, val[0] + val[1]*i, 10, 10, 0), vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 1000)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 1000)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)
_ = cp.sensitivity([g1_1,g1_2,st, so], f, [(.1, .899)], [(.1, .099)], T, [l1, l2, l3, l7], [np.zeros(T)])

**3.1 teszt:**

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 750, 750, 1000), vo.VPPGasEngine(1500, 1500, 1500, 1000), vo.VPPEnergyStorage(0, 10000, 1000, 1000, val, val, 10, 10, 0), vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
cp.sensitivity_by_participants(f, 0.01, 0.5+0.499*1, 3, l1, d, T)

**3.2 teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 1000), vo.VPPGasEngine(1500, 1500, 1500, 1000), vo.VPPEnergyStorage(0, 10000, 1000, 1000, val[0] + val[1]*i, val[0] + val[1]*i, 10, 10, 0), vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 1000)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 1000)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)
_ = cp.sensitivity([g1_1,g1_2,st, so], f, [(.1, .899)], [(.1, .099)], T, [l1, l2, l3, l7], [np.zeros(T)])

**3.2.1 Teszt:**

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 750, 750, 1000), vo.VPPGasEngine(1500, 1500, 1500, 1000), vo.VPPEnergyStorage(0, 10000, 1000, 1000, val, val, 10, 10, 0), vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)]
cp.sensitivity_by_participants(f, 0.01, 0.5+0.499*1, 3, l1, d, T)

**3.2.1 Teszt:**

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 750, 750, 1000), vo.VPPGasEngine(1500, 1500, 1500, 1000), vo.VPPEnergyStorage(0, 10000, 1000, 1000, val, val, 10, 10, 0), vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)]
cp.sensitivity_by_participants(f, 0.01, 0.5+0.499*1, 3, l7, d, T)

## MILP solver

### Fel- és lefutás érzékenysége (Gázmotoroknál)

#### Csak gázmotorok

**1. Teszt:**<br>
2-2 gázmotorunk van, amelyek közül a második pár 2. tagjánál módosítjuk a ramp up/down rate-et.
Csak az *l1, l2, l3, l7* fogyaztói görbéket használjuk, mivel a többi nem teljesíthető. Ezen teszt esetén a ramp up és down értékeket csak csökkentjük, növelés nem történik.

**Eredmény:**

In [ ]:
cp.globalcurrcat = 0
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)


_ = cp.sensitivity([g1_1,g1_2], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Az 1. teszt módosított verziója, hogy a 3. vizsgálat is teljes egészében optimalizálható legyen.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)


_ = cp.sensitivity([g1_1,g1_2], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 625),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2 Teszt:**<br>
Még egy kisebb módosítás, amely a költségeket érinti. Jelen esetben meg lett 10-szerezve az első gázmotor költsége és az olcsóbb gázmotor ramp up/down értékeit csökkentjük.<br>
**Eredmény:** Ezzel a módosítással már elég érdekes eremények születtek. Ahogy látható, akár az első tesz esetén is már a költség függvény változásnak indult. Ez a változás ránézésre a konstans 0-ból egy exponenciális függvény.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)

_ = cp.sensitivity([g1_1,g1_2], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 625),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.3. Teszt:**<br>
Annyi módosítással az 1.2-höz képest, hogy 1500 helyett minden gázmotornak 750 az alap ramp up/down értéke.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
              vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)

_ = cp.sensitivity([g1_1,g1_2], f, [(750, 750)], [(750, 593),(750, 750)], T, [l1, l7], [np.zeros(T)])

**1.3.1 Teszt:**<br>
Az elöző teszteset részletesebb vizsgálata. (1. eset csökkentés)

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 750, 750, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val, val, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
cp.sensitivity_by_participants(f, 190, 750, 10, l1, d, T)

**2. Teszt**<br>
Növeljük meg a maximális termelésüket a motoroknak és ezzel az alap ramp up/down rátát is.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)

_ = cp.sensitivity([g1_1,g1_2], f, [(2000, 4000)], [(2000, 2000),(2000, 2000),(2000, 1677),(2000, 2000),(2000, 2000),(2000, 2000),(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.1 Teszt:**<br>
Legyen drágább a nagyobb max. termeléssel bíró gázmotor.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
              vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)

_ = cp.sensitivity([g1_1,g1_2], f, [(2000, 4000)], [(2000, 2000),(2000, 2000),(2000, 1677),(2000, 2000),(2000, 2000),(2000, 2000),(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.2 Teszt:**<br>
Most a másik motor legyen drágább.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
              vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)

_ = cp.sensitivity([g1_1,g1_2], f, [(2000, 4000)], [(2000, 2000),(2000, 2000),(2000, 1677),(2000, 2000),(2000, 2000),(2000, 2000),(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**Összefogalaló:**<br>
Az előbbi tesztek esetén amennyiben azonos költségekkel futottak a motorok azon esetekben a költség nem változott, viszont amint különbség volt közöttük, egyből exponeneciális változás volt tapasztalható.
Termelés tekintetében is a költséghez van köze a függvény alakjának, ha a költségek megegyeznek, akkor egy keszekusza függvényt kapunk, viszont amint különböznek, ez is átvált exponenciálisba.

#### Gázmotorok + energia tároló

**1. Teszt:**<br>
Úgy, hogy bejött a tároló, az eddig nem teljesíthető optimalizáciokat is el lehetett végezni, viszont ennek következtében az ottani költségek hatalmasat ugrottak.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
              vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
                vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1,g1_2, t], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Visszatérve a tároló nélkül is teljesíthető paraméterekre. Ezen eredmények igazolják az előbbi tesztnél leírtakat.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
              vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
                vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)


_ = cp.sensitivity([g1_1,g1_2, t], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 625),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2 Teszt:**<br>
Növeljük meg az egyik gázmotor költségét.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
              vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
              vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)

_ = cp.sensitivity([g1_1,g1_2,t], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 625),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2.1 Teszt:**<br>
Most helyezzük vissza az összes paramétert úgy, hogy azok 1500-nak százalékos értékével csökkentsék az alap értéket. Ez nem fogja <br>
**Eredmény:** Nem tapasztalható releváns eltérés a grafikonok között. Ránézésre olyan, mintha nagyobb intervallumon vennénk a függvényeket.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)

_ = cp.sensitivity([g1_1,g1_2,t], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.3. Teszt:**<br>
Csökentjük az alap ramp up/down értékeket. (1500 $\rightarrow$ 750)

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)

_ = cp.sensitivity([g1_1,g1_2,t], f, [(750, 750)], [(750, 750)], T, [l1, l2, l7], [np.zeros(T)])

**2. Teszt**<br>
Növeljük meg a maximális termelésüket a motoroknak és ezzel az alap ramp up/down rátát is.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)

_ = cp.sensitivity([g1_1,g1_2,t], f, [(2000, 4000)], [(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.1 Teszt:**<br>

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)

_ = cp.sensitivity([g1_1,g1_2,t], f, [(2000, 4000)], [(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.2 Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10)

_ = cp.sensitivity([g1_1,g1_2,t], f, [(2000, 4000)], [(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**Összefoglalás:**<br>
Összességében hasonló görbéket kapunk, mintha csak gázmotorok szerepelnének, csak kevésbé szögleteseket, a tárolónak köszönhetően sokkal "görbébb" függvényeket kapunk. Ezek mellett sokszor kisebbek a hibák, mintha csak gázmotorok lennének, illetve sok esetben az eddig nem teljesíthető teszteket is sikerül elvégeznie.

#### Gázmtorok + napelem

**1. Teszt:**<br>
Adjunk egy napelemet a 2. gázmotorhoz, úgy, hogy annak termelését arányosra állítjuk a gázmotorokéval. 

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Módosítjuk a paraétereket, hogy minden esetben elvégezhető legyen az optimalizálás.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 945),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2 Teszt:**<br>
Kis költség változtatás.<br>
**Eredmény:** Hasonló az eredmény, mint az eddigieknél. (Exponenciális költség növekedés.)

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 945),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2.1 Teszt:**<br>
Paraméterek visszaállítása az 1. tesztbeli értékekre.
Itt most nincs különbség a kettő között, nem mint a tárolók esetén...

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2.2 Teszt:**<br>
Játszunk a költséggel. Lehet, hogy azért exponeneciális a költség különbség függvénye, mert a költségek is így viszonyulnak egymáshoz?<br>
**Eredmény:** Az alábbi grafikonok alapján téves a feltevés, mivel így is exponeneciálisak lesznek.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 20, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 20, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(1500, 0)], [(1500, 1500),(1500, 1165),(1500, 945),(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.3 Teszt:**<br>
Módosítsuk az alap ramp up/down értékeket az eddigi felére. (1500 $\rightarrow$ 750)

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(750, 750)], [(750, 750)], T, [l1, l7], [np.zeros(T)])

**2. Teszt:**<br>
Legyen drágább a nagyobb max. termeléssel bíró gázmotor.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(2000, 4000)], [(2000, 2000),(2000, 2000),(2000, 1997),(2000, 2000),(2000, 2000),(2000, 2000),(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.1 Teszt:**<br>
Legyen drágább a nagyobb max. termeléssel bíró gázmotor.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(2000, 4000)], [(2000, 2000),(2000, 2000),(2000, 1997),(2000, 2000),(2000, 2000),(2000, 2000),(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.2 Teszt:**<br>
Most a másik motor legyen drágább.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(2000, 4000)], [(2000, 2000),(2000, 2000),(2000, 1997),(2000, 2000),(2000, 2000),(2000, 2000),(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**Összefogalalás:**
Többnyire hasonló görbéket kapunk, mint a sima gázotoroknál, csak a legtöbb esetben sokkal kacifántosabbak. Ezen felül előfordul olyan eset, melyet ez a modell teljesíteni tud, míg a sima gázmotoros nem, mely várható volt a termelt energia mennyiségének növekedése végett.

#### Gázmotork + napelem + enegia tároló

**1. Teszt:**<br>
Az eddigi résztvevőket kombinálva most 2 gázmotorral, 1 tárolóval és 1 napelemmel fogjuk az optimalizációt végezni.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0), 
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)


_ = cp.sensitivity([g1_1,g1_2, t, s], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Megváltoztatjuk a költségeket a gázmotorok esetén.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0), 
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)


_ = cp.sensitivity([g1_1,g1_2, t, s], f, [(1500, 0)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.2 Teszt:**<br>
Módosítsuk az alap ramp up/down értékeket az eddigi felére. (1500 $\rightarrow$ 750)

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0), 
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)


_ = cp.sensitivity([g1_1,g1_2, t, s], f, [(750, 750)], [(750, 750)], T, [l1, l2, l3, l7], [np.zeros(T)])

**2. Teszt:**<br>
Legyen drágább a nagyobb max. termeléssel bíró gázmotor.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0), 
             vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
s = vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)


_ = cp.sensitivity([g1_1,g1_2, t, s], f, [(2000, 4000)], [(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.1 Teszt:**<br>
Legyen drágább a nagyobb max. termeléssel bíró gázmotor.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0), 
             vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
s = vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)


_ = cp.sensitivity([g1_1,g1_2, t, s], f, [(2000, 4000)], [(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**2.2 Teszt:**<br>
Most a másik motor legyen drágább.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(4000, 2000, 2000, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(6000, val[0]+val[1]*i, val[0]+val[1]*i, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0), 
             vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(4000, 2000, 2000, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(6000, 2000, 2000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
t = vo.VPPEnergyStorage(0,10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
s = vo.VPPSolarPanel(2000, 1000, 1000, 1, T, seed=seed)


_ = cp.sensitivity([g1_1,g1_2, t, s], f, [(2000, 4000)], [(2000, 2000)], T, [l1, l2,l3,l5,l6, l7,l10], [np.zeros(T)])

**Összefogalalás:**
Itt is többnyire hasonló görbéket kapunk, mint a tárolók esetén, viszont még ívesebbeket, kevesebb "hegye" van a görbéknek. Ezen felül ebben az esetben a hibák hamarabb kisebb változtatások esetén is már kezdenek előjönni, viszont kisebbek, mint a többi esetén. Hasonló mondható el a költségekről is, hamarabb nőnek, viszont sokkal kevesebbet.

### Költség érzékenysége (Gázmotoroknál)

#### Csak gázmotorok

**1. Teszt:** <br>
2-2 gázotorral végezzük ezt a tesztet és azok közül az egyik költségét változtatjuk.<br>
**Eredmény:** Ahogy azt a grafikonokon is láthatjuk a paraméter változtatására vagy szakaszonként, vagy telejsen lineáris változást tapasztalhatunk a költségfüggvényben.

In [ ]:
cp.globalcurrcat = 1
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, val[0]+i*val[1], None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)


_ = cp.sensitivity([g1_1,g1_2], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Vizsgáljuk meg azt az esetet, amikor a változtatott költségű gázmotor fel-és lefutási értékeit megfelezzük.<br>
**Eredmény:** Sok különbség az elöző esethez képest nincsen a költségeket tekintve, viszont itt már a termelésben észlelhetünk különbségeket, de többniyre konstans hibákkal. Ezen felül a 3. teszteset itt már nem teljesíthető.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, val[0]+i*val[1], None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)


_ = cp.sensitivity([g1_1,g1_2], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**2. Teszt:**<br>
Most növeljük meg a gázmotorok maximális termelésést és vele együtt a fel- és lefutási ráták értékeit.<br>
**Eredmény:** sok különbség ez esetben nincs az eddigi ereményekhez képest. Egyedül az hozható fel, hogy több esetet képes teljesíteni.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(6000, 3000, 3000, 500, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(4000, 2000, 2000, val[0]+i*val[1], None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
g1_1 = vo.VPPGasEngine(6000, 3000, 3000, 500, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(4000, 2000, 2000, 500, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)


_ = cp.sensitivity([g1_1,g1_2], f, [(500, 500)], [(500, 500)], T, [l1, l2, l3, l5, l6, l7, l10], [np.zeros(T)])

#### Gázmotorok + tároló 

**1. Teszt:** <br>
A teszt során 2 gázmotor és egy energia tároló segítségével oldjuk meg a feladatokat. <br>
**Eredmények:** Az elöző szekció 1 tesztjéhez hasonlítva itt jelentős eltérések vehető észre. Ilyen például a 2. teszt esetén 50-55% közötti érdekes ugrás a termelések hibájában, melyet a következő teszt során méylebben megvizsgálok. Összességében viszont az mondható el, hogy olcsóbbban tereli meg a modell a szükséges áramot. 

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, val[0]+i*val[1], None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1, g1_2, s], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Ezen teszt során egy megadott intervallumon változtatva a költség paraméterét részletesen, részvevőkre lebontva elemezem a termelést.<br>
**Eredmények:** Megvizsgálva a termelési görbéket szépen látszik, ahogy a költséget változtatjuk azok miként alakulnak. Ezen az intervallumon 3 ponton történik változás, melyek rendre 17 és 18 között, 20 és 21 között, valamint 22 és 23 között. Ha összevetjük az elöző teszt 2. esetével jól látszik, hogy ahogyan a költség függvény változik ott miként történik a termelésben


In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, val, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50-50*0.5, 50-50*0.4, 10, l2, d, T)

**1.2 Teszt:**<br>
Másik intervallumra is megnézve.<br>
**Eredmények:** Itt csak 1 töréspont vehető észre.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, val, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50+50*0.6,50+50*0.7, 10, l2, d, T)

**1.3 Teszt:**<br>
Nézzük meg másik fogyasztási görbére is.<br>
**Eredmények:** Itt is csak 1 ilyen pont található.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, val, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50+50*0.65,50+50*0.75, 10, l3, d, T, (30,150))

**2. Teszt:**<br>
Vizsgáljuk meg azt az esetet, amikor a változtatott költségű gázmotor fel-és lefutási értékeit megfelezzük<br>
**Eredmények:** A sima gázmotoros esethez képest sokkal érdekesebb eredményeket kaptunk. Sok esetben nőttek azon pontok száma ami után a modell termelési stratégiát vált. Következő tesztek ezek részletesebb vizsgálatáról fognak szólni.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 750, 750, val[0]+i*val[1], None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1, g1_2, s], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**2.1 Teszt:**<br>
Ezöző teszt 1. vizsgálatának részletesebb megtekintése.<br>
**Eredmények:** Szépen látszik a grafikonon, hogy egy idő után már érdemes bekapcsolni az 1-es gázmotort, mert úgy olcsóbb a termelés.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 750, 750, val, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50-50*0.45,50-50*0.5, 10, l1, d, T, (30,40), addcontour=False)

**2.2 Teszt:**<br>
Ezöző teszt 2. vizsgálatának részletesebb megtekintése.<br>
**Eredmények:** It is szépen látszik a grafikonokon, hogy hol változnak az értékek itt is a változások száma megegyezik a grafikonon láthatóéval.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 750, 750, val, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50-50*0.55,50-50*0.25, 10, l2, d, T, (30,40), addcontour=False)

**3. Teszt:**<br>
Növeljük meg a termelési képességüket a gázmotoroknak.<br>
**Eredmények:** Az eredményekben nincs semmi érdekes. A hibák konstansok, a költség lineárisan csökken/nő.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(6000, 3000, 3000, 500, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(4000, 2000, 2000, val[0]+i*val[1], None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPEnergyStorage(0, 10000, 2000, 2000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(6000, 3000, 3000, 500, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(4000, 2000, 2000, 500, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPEnergyStorage(0, 10000, 2000, 2000, 0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1,g1_2,s], f, [(500, 500)], [(500, 500)], T, [l1, l2, l3, l5, l6, l7, l10], [np.zeros(T)])

#### Gázmotorok + napelem

**1. Teszt:**<br>
Ezen tesztek során 2 gázmotor és egy napelem lesz a részvevőnk.<br>
**Eredmények:** A grafikonokon 1 érdekes jelenség van csak. A csökkentés esetén az utolcsó -100%-ra a hiba megugrik.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, val[0]+val[1]*i, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Most az elöző teszt esetén említett felugrást vizsgálom.<br>
**Eredmények:** Mivel addigra a gázmotor költsége 0 lesz, így inkább kitermeli ingyen az energiát, minthogy a napelemét használja 1 költségért.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, val, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
cp.sensitivity_by_participants(f, 0,50-50*0.95, 2, l1, d, T)

**2. Teszt:**<br>
Módosítsuk most a változtatott gázmotor fel- és lefutási értékeit a felére.<br>
**Eredmények:** Hasonló eseteket tapasztalhatunk mintha csak gázmotorok lennének a teszben. Viszont a termelési hibák amikor változnak akkor azt kisebb értékben teszik.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 750, 750, val[0]+val[1]*i, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**3. Teszt:**<br>
Növeljük meg a termelési képességüket a gázmotoroknak.<br>
**Eredmények:** Nincs semmi érdekes sajnos. Ugyan azt kaptuk, mint az eredeti gázmotorsnál, csak itt vannak hibák is és olcsóbb a termelés a napelem miatt.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(6000, 3000, 3000, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(4000, 2000, 2000, val[0]+val[1]*i, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(6000, 3000, 3000, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(4000, 2000, 2000, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, s], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

#### Gázmotorok + napelem + enegia tároló

**1. Teszt:**<br>
A tesztek során 2 gázmotort, 1 tárolót és 1 napelemet alkalmazok.<br>
**Eredmények:** Az első tesztnél már látható, hogy a terlésnél a hiba eléggé hullámos, amit a következő tesztekben jobban meg is vizsgálok.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, val[0]+val[1]*i, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed),
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, so, st], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Az elöző teszteset első vizsgálatának részletesebb tárgyalása.<br>

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, val, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed),
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50-50*0.8,50-50*0.4, 4, l1, d, T)

**2. Teszt:**<br>
Módosítsuk a változtatott gázmotor fel- és lefutási rátáját a felére.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 750, 750, val[0]+val[1]*i, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed),
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, so, st], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l7], [np.zeros(T)])

**2.1 Teszt:**<br>
Vizsgáljuk meg a 3. esetet az elöző tesztből.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 750, 750, val, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed),
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50-50*0.55,50-50*0.25, 4, l3, d, T)

**3. Teszt:**<br>
Növeljük meg a gázmotorok termelését.

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(6000, 3000, 3000, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(4000, 2000, 2000, val[0]+val[1]*i, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed),
             vo.VPPEnergyStorage(0, 10000, 3000, 3000, 0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(6000, 3000, 3000, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(4000, 2000, 2000, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
st = vo.VPPEnergyStorage(0, 10000, 3000, 3000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2, so, st], f, [(50, 50)], [(50, 50)], T, [l1, l2, l3, l4, l5, l6, l7, l10], [np.zeros(T)])

**3.1 Teszt:**<br>
Vizsgáljuk meg a 2. esetet az elöző tesztből.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(6000, 3000, 3000, 50, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(4000, 2000, 2000, val, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed),
             vo.VPPEnergyStorage(0, 10000, 3000, 3000, 0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 50,50+50*0.05, 3, l2, d, T)

### Max termlés érzékenysége (Gázmotoroknál)

#### Csak gázmotorok

**1. Teszt:**

In [ ]:
cp.globalcurrcat = 2
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val[0]+val[1]*i, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)


_ = cp.sensitivity([g1_1,g1_2], f, [(1500, 1500)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Elöző teszt 2 eset pozitív irányának vizsgálata.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
cp.sensitivity_by_participants(f, 1500- 1500*0.5,15001500*0.3, 3, l1, d, T, addcontour=False)

**2. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val[0]+val[1]*i, 3000, 3000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 3000, 3000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)


_ = cp.sensitivity([g1_1,g1_2], f, [(1500, 7000)], [(1500, 1500)], T, [l1, l2, l3,l7], [np.zeros(T)])

**2.1 Teszt:**<br>
Elöző teszt 1 esete csökkentésnél.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val, 3000, 3000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
cp.sensitivity_by_participants(f, 0, 1500-1500*0.35, 10, l1, d, T)

**2.2 Teszt:**<br>
Elöző teszt 2 esete növelésnél.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val, 3000, 3000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
cp.sensitivity_by_participants(f, 1500, 1500+7000*0.1, 10, l2, d, T)

**2.3 Teszt:**<br>
Elöző teszt 3 esete növelésnél.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val, 3000, 3000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)]
cp.sensitivity_by_participants(f, 1500, 1500+7000*0.2, 10, l3, d, T)

#### Gázmotorok + tároló

**1. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val[0]+val[1]*i, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0,10000, 1000, 1000 ,0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPEnergyStorage(0,10000, 1000, 1000 ,0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1,g1_2,s], f, [(1500, 1500)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Elöző teszt 1 eset negatív irány.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0,10000, 1000, 1000 ,0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 1500, 1500+1500*0.05, 3, l2, d, T)

**1.2 Teszt:**<br>
Elöző teszt 2 eset negatív irány.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0,10000, 1000, 1000 ,0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 1500-1500*0.65, 1500-1500*0.45, 3, l2, d, T)

**1.2.1 Teszt:**<br>
Elöző teszt 2 eset negatív irány másik intervallumon.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0,10000, 1000, 1000 ,0.1, 0.1, 10, 10, 0)]
cp.sensitivity_by_participants(f, 1500-1500*0.85, 1500-1500*0.65, 10, l2, d, T)

**2. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val[0]+val[1]*i, 3000, 3000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0,10000, 1000, 1000 ,0.1, 0.1, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 3000, 3000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
s = vo.VPPEnergyStorage(0,10000, 3000, 3000 ,0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1,g1_2,s], f, [(1500, 7000)], [(1500, 1500)], T, [l1, l2, l3,l7], [np.zeros(T)])

#### Gázmotorok + napelem

**1. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val[0]+val[1]*i, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2,so], f, [(1500, 1500)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1. Teszt:**<br>
Elöző teszt 1 estének negatív oldala.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
cp.sensitivity_by_participants(f, 1500-1500*0.4, 1500-1500*0.3, 3, l1, d, T)

**2. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val[0]+val[1]*i, 3000, 3000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 3000, 3000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2,s], f, [(1500, 7000)], [(1500, 1500)], T, [l1, l2, l3,l7], [np.zeros(T)])

#### Gázmotorok + tároló + napelem

**1. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val[0]+val[1]*i, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0),
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2,st, so], f, [(1500, 1500)], [(1500, 1500)], T, [l1, l2, l3, l7], [np.zeros(T)])

**2. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(val[0]+val[1]*i, 3000, 3000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1),
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0),
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 1500, 1500, 100, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 3000, 3000, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)

_ = cp.sensitivity([g1_1,g1_2,st, so], f, [(1500, 7000)], [(1500, 1500)], T, [l1, l2, l3,l7], [np.zeros(T)])

### Veszteségi faktor érzékenysége (Tárolók)

**1. Teszt:**

In [ ]:
cp.globalcurrcat = 3
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, val[0] + val[1]*i, val[0] + val[1]*i, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1,g1_2,st], f, [(.1, .899)], [(.1, .099)], T, [l1, l2, l3, l7], [np.zeros(T)])

**1.1 Teszt:**<br>
Elöző teszt 3. esetének részletesebb vizsgálata.

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 750, 750, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, 10, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, val, val, 10, 10, 0)]
cp.sensitivity_by_participants(f, 0.01, 0.5+0.499*1, 3, l3, d, T)

**2. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, val[0] + val[1]*i, val[0] + val[1]*i, 10, 10, 0)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)

_ = cp.sensitivity([g1_1,g1_2,st], f, [(.1, .899)], [(.1, .099)], T, [l1, l2, l3, l7], [np.zeros(T)])

**3. Teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, val[0] + val[1]*i, val[0] + val[1]*i, 10, 10, 0), 
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)
_ = cp.sensitivity([g1_1,g1_2,st, so], f, [(.1, .899)], [(.1, .099)], T, [l1, l2, l3, l7], [np.zeros(T)])

**3.1 teszt:**

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 750, 750, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, val, val, 10, 10, 0), 
             vo.VPPSolarPanel(1000, 500, 500, 1, T, seed=seed)]
cp.sensitivity_by_participants(f, 0.01, 0.5+0.499*1, 3, l1, d, T)

**3.2 teszt:**

In [ ]:
def f(val, i):
     return [vo.VPPGasEngine(1500, 750, 750, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, val[0] + val[1]*i, val[0] + val[1]*i, 10, 10, 0), 
             vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
g1_2 = vo.VPPGasEngine(1500, 1500, 1500, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1)
st = vo.VPPEnergyStorage(0, 10000, 1000, 1000, 0.1, 0.1, 10, 10, 0)
so = vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)
_ = cp.sensitivity([g1_1,g1_2,st, so], f, [(.1, .899)], [(.1, .099)], T, [l1, l2, l3, l7], [np.zeros(T)])

**3.2.1 Teszt:**

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 750, 750, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, val, val, 10, 10, 0), 
             vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)]
cp.sensitivity_by_participants(f, 0.01, 0.5+0.499*1, 3, l1, d, T)

**3.2.1 Teszt:**

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 750, 750, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPGasEngine(1500, 1500, 1500, 1000, None, 20, 2000, 2000, 2, 2, 2, 2, go0=1), 
             vo.VPPEnergyStorage(0, 10000, 1000, 1000, val, val, 10, 10, 0), 
             vo.VPPSolarPanel(3000, 1500, 1500, 1, T, seed=seed)]
cp.sensitivity_by_participants(f, 0.01, 0.5+0.499*1, 3, l7, d, T)

# Összegzés

## Ez a kódrész csak akkor ad értelmes adatokat, ha előtte futtatva lettek a fentebbi tesztek úgy, hogy a `globalsummerize` kapcsoló igazra volt állítva (ilyen esetben ajánlom a `globaldisplay` kapcsoló hamisra állítását, hogy a grafikonok ne jelenjenek meg).

In [ ]:
cp.summerize()

In [ ]:
def f(val, i, j):
     return [vo.VPPGasEngine(1500, 750, 750, 100), vo.VPPGasEngine(1500, val[0]+val[0]*i, val[0]+val[0]*i, val[1]+val[1]*j)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 100)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 50)

_ = cp.sensitivity_2([g1_1,g1_2], f, ["Fel-, lefutási ráta", "Költség"], [(750, 50)], T, [l1, l7], [np.zeros(T)], view=[[(30,315), (30,315), (30,315)]])

In [ ]:
def f(val, i, j):
     return [vo.VPPGasEngine(1500, 750, 750, 100, None, 20, 1000, 1000, 2, 2, 2, 2), vo.VPPGasEngine(1500, val[0]+val[0]*i, val[0]+val[0]*i, val[1]+val[1]*j, None, 20, 1000, 1000, 2, 2, 2, 2)]
g1_1 = vo.VPPGasEngine(1500, 750, 750, 100, None, 20, 1000, 1000, 2, 2, 2, 2)
g1_2 = vo.VPPGasEngine(1500, 750, 750, 50, None, 20, 1000, 1000, 2, 2, 2, 2)

_ = cp.sensitivity_2([g1_1,g1_2], f, ["Fel-, lefutási ráta", "Költség"], [(750, 50)], T, [l1, l7], [np.zeros(T)], view=[[(30,315), (30,315), (30,315)]])

In [ ]:
def f(val):
     return [vo.VPPGasEngine(1500, 750, 750, 100), vo.VPPGasEngine(1500, 1500, 1500, val)]


cp.sensitivity_by_participants(f, 99, 100, 4, l1, d, T)